In [16]:
import os
from azureml.core import Model, Workspace
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.pipeline.wrapper import PipelineRun
from azureml.core.compute import AmlCompute, AksCompute, ComputeTarget
from azureml.core.webservice import LocalWebservice, AciWebservice, Webservice, AksWebservice

In [2]:
subscription_id = '4f455bd0-f95a-4b7d-8d08-078611508e0b'
resource_group = 'fundamental'
workspace_name = 'fundamental3'
tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47"
# for loading module
namespace=workspace_name
InteractiveLoginAuthentication(tenant_id=tenant_id)
workspace = Workspace(subscription_id, resource_group, workspace_name)
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, workspace.compute_targets.keys(),sep = '\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


fundamental3
fundamental
eastasia
4f455bd0-f95a-4b7d-8d08-078611508e0b
dict_keys(['myaks2', 'aml-compute', 'my-compute', 'compute-deploy'])


In [3]:
# get a PipelineRun object
experiment_name = "deploy"
experiment = Experiment(workspace, experiment_name)
run_id = '24f0ba77-1ef8-4a9a-b9a8-9ed14d98190e'
pipeline_run = PipelineRun(experiment, run_id)

In [4]:
# get a StepRun object
step_run = pipeline_run.find_step_run(name = 'FastText Train')[0]

In [10]:
# download model from the output port of the Train module
port = step_run.get_port(name='Trained model dir')
saved_path = port.download(local_path='data', overwrite=True)
print('model save at: {}'.format(saved_path))

Downloaded azureml/358a3e99-f299-4089-b2cf-cecc32ac34f8/Trained_model_dir/BestModel, 1 files out of an estimated total of 1
model save at: data\azureml/358a3e99-f299-4089-b2cf-cecc32ac34f8/Trained_model_dir


In [11]:
# register model for deployment
model_name = os.listdir(saved_path)[0]
model_path = os.path.join(saved_path, model_name)
model = Model.register(workspace, model_path=model_path, model_name='model_for_deploy', tags={'deploy':1})

Registering model model_for_deploy


In [12]:
env_list = Environment.list(workspace)
name='env_for_deployment'
if name not in env_list:
    file_path='deployment/env_for_deployment.yaml'
    env = Environment.from_conda_specification(name=name, file_path=file_path)
    env = env.register(workspace=workspace)
else:
    env = Environment.get(workspace=workspace, name=name)

In [13]:
# define inference configuration
entry_script='scoring_for_deployment.py'
source_directory='deployment'
inference_config = InferenceConfig(entry_script=entry_script, source_directory=source_directory,
                                       environment=env)

In [18]:
# deploy locally
service_name='local-deploy-test'
models=[model]
port=8892
deployment_config = LocalWebservice.deploy_configuration(port=port)
service_locally = Model.deploy(workspace=workspace, name=service_name, models=models, inference_config=inference_config,
                       deployment_config=deployment_config)
service_locally.wait_for_deployment(show_output=True)
print(service_locally.state)
print(service_locally.get_logs())

In [ ]:
# deploy to ACI (Azure Container Instances)
# every time we deploy to ACI, we need to change the service_name or we delete the existing service beforehand
service_name='aci-deploy-test7'
models=[model]
overwrite=True
cpu_cores=1
memory_gb=1
services = workspace.webservices
if service_name in services and overwrite:
    print('found existing service named {}, delete it right now...'.format(service_name))
    services[service_name].delete()
deployment_config = AciWebservice.deploy_configuration(cpu_cores=cpu_cores, memory_gb=memory_gb)
service_aci = Model.deploy(workspace, service_name, models=models, inference_config=inference_config,
                       deployment_config=deployment_config)
service_aci.wait_for_deployment(show_output=True)
print(service_aci.state)

found existing service named aci-deploy-test7, delete it right now...
Running.....................................................................................................................................................................................................

In [13]:
# deploy to AKS (Azure Kubernetes Service)
# workspace with AKS
aks_subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22'
aks_resource_group = 'DesignerDRI'
aks_workspace_name = 'DesignerDRI_EASTUS'
aks_workspace = Workspace(aks_subscription_id, aks_resource_group, aks_workspace_name)

name: DesignerDRI_EASTUS
resource_group DesignerDRI
location eastus
subscription_id 74eccef0-4b8d-4f83-b5f9-fa100d155b22
compute_targets dict_keys(['attachedcompute', 'myaks1', 'default', 'compute', 'cpu-cluster', 'aml-compute'])
Registering model model_for_deploy


In [ ]:
# register model in this workspace
model = Model.register(aks_workspace, model_path=model_path, model_name='model_for_deploy', tags={'deploy':1})

In [14]:
# deploy to AKS (Azure Kubernetes Service)
attachment_name='myaks1'
# every time we deploy to ACI, we need to change the service_name or we delete the existing service beforehand
service_name='aks-deploy-test7'
models=[model]
cpu_cores=1
memory_gb=1
token_auth_enabled=True
services = workspace.webservices
if service_name in services and overwrite:
    print('found existing service named {}, delete it right now...'.format(service_name))
    services[service_name].delete()
# Only one type of Auth may be enabled
if token_auth_enabled:
    # key auth
    auth_enabled = False
else:
    auth_enabled = True
print('auth type: {}'.format('token' if token_auth_enabled else 'key'))
aks_target = AksCompute(aks_workspace, attachment_name)
# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores=cpu_cores, memory_gb=memory_gb,
                                                       token_auth_enabled=token_auth_enabled,
                                                       auth_enabled=auth_enabled)
service = Model.deploy(workspace, service_name, models, inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output=True)
print(service.state)

found existing service named aks-deploy-test7, delete it right now...
auth type: token
Running............................................................
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 79909e99-c1db-4574-95bc-edb3d83ce7d8
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesError",
  "statusCode": 400,
  "message": "Kubernetes Deployment Error",
  "details": [
    {
      "code": "Unschedulable",
      "message": "0/3 nodes are available: 3 Insufficient cpu."
    },
    {
      "code": "DeploymentFailed",
      "message": "Couldn't schedule because the kubernetes cluster didn't have available resources after trying for 00:05:00.\nYou can address this error by either adding more nodes, changing the SKU of your nodes or changing the resource requirements of your service.\nPlease refer to https://aka.ms/debugimage#container-cannot-be-scheduled for more information."
    }
  ]
}

Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 79909e99-c1db-4574-95bc-edb3d83ce7d8
More informa

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 79909e99-c1db-4574-95bc-edb3d83ce7d8
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesError",
  "statusCode": 400,
  "message": "Kubernetes Deployment Error",
  "details": [
    {
      "code": "Unschedulable",
      "message": "0/3 nodes are available: 3 Insufficient cpu."
    },
    {
      "code": "DeploymentFailed",
      "message": "Couldn't schedule because the kubernetes cluster didn't have available resources after trying for 00:05:00.\nYou can address this error by either adding more nodes, changing the SKU of your nodes or changing the resource requirements of your service.\nPlease refer to https://aka.ms/debugimage#container-cannot-be-scheduled for more information."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached